In [14]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [15]:

import folium
import json
from folium import plugins

In [3]:
import pandas as pd
import glob
import os
import plotly.express as px



In [4]:
#glob and pandas

path = r'C:\ppp_data' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)



In [7]:
len(frame)

4224170

In [8]:
#EXAMPLE newdf = df.loc[df['column'] == match #remember, string or int  # Comment


dfbars = frame.loc[frame['NAICSCode'] == 722310]  #bars
dfmobile = frame.loc[frame['NAICSCode'] == 722330]  #mobile food 
dffsr = frame.loc[frame['NAICSCode'] == 722511]  #Full-Service Restaurants
dfqsr = frame.loc[frame['NAICSCode'] == 722513]  #Limited-Service Restaurants
dfcaf = frame.loc[frame['NAICSCode'] == 722514]  #Cafeterias, Grill Buffets, and Buffet
dfsnk = frame.loc[frame['NAICSCode'] == 722515]  #Snacks and coffee



In [46]:
#dfbars = frame.loc[frame['NAICSCode'] == '722310']  #bars


In [9]:
dfList = [dfbars, dfmobile, dffsr, dfqsr, dfcaf, dfsnk]  # List of your dataframes
allrest = pd.concat(dfList)

In [41]:
print('Bars: ' + str(len(dfbars)))
print('Mobile: ' + str(len(dfmobile)))
print('Full-Service: ' + str(len(dffsr)))
print('QSR: ' + str(len(dfqsr)))
print('Cafeteria: ' + str(len(dfcaf)))
print('Snack & Cafe: ' + str(len(dfsnk)))
print('All Restaurants: ' + str(len(allrest)))

Bars: 7553
Mobile: 3729
Full-Service: 136863
QSR: 67806
Cafeteria: 2102
Snack & Cafe: 12501
All Restaurants: 230554


In [45]:
#allrest.to_csv('file_name.csv’) # relative position

               
allrest.to_csv (r'C:\ppp_data\export_dataframe.csv', index = False, header=True)


In [12]:
# df = px.data.gapminder().query("year==2007")
# fig = px.scatter_geo(df, locations="iso_alpha", color="continent",
#                      hover_name="country", size="pop",
#                       projection="natural earth")
# fig.show()

ValueError: 
    Invalid value of type 'builtins.str' received for the 'type' property of layout.geo.projection
        Received value: 'north america'

    The 'type' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            ['equirectangular', 'mercator', 'orthographic', 'natural
            earth', 'kavrayskiy7', 'miller', 'robinson', 'eckert4',
            'azimuthal equal area', 'azimuthal equidistant', 'conic
            equal area', 'conic conformal', 'conic equidistant',
            'gnomonic', 'stereographic', 'mollweide', 'hammer',
            'transverse mercator', 'albers usa', 'winkel tripel',
            'aitoff', 'sinusoidal']

In [34]:


with open('zip_codes.json') as f:
    laArea = json.load(f)
    
   # C:/Documents/Untitled%20Folder/zip_codes.json

In [40]:
frame.head

<bound method NDFrame.head of          LoanAmount       City State      Zip  NAICSCode  \
0          149927.0  FAIRBANKS    AK  99709.0   541110.0   
1          149577.0     JUNEAU    AK  99801.0   238220.0   
2          149465.0  ANCHORAGE    AK  99501.0   813990.0   
3          149300.0  ANCHORAGE    AK  99501.0   488190.0   
4          149000.0  KETCHIKAN    AK  99901.0   541211.0   
...             ...        ...   ...      ...        ...   
4224165       400.0     LANDER    WY  82520.0   722513.0   
4224166       400.0     CASPER    WY  82601.0   453220.0   
4224167       390.0     LOVELL    WY  82431.0   561790.0   
4224168       377.3     CASPER    WY  82604.0   624410.0   
4224169       260.0     CASPER    WY  82609.0   623990.0   

                            BusinessType RaceEthnicity        Gender  \
0                    Sole Proprietorship    Unanswered    Male Owned   
1                            Corporation    Unanswered    Unanswered   
2                Non-Profit Organ

In [39]:
#group the starbucks dataframe by zip code and count the number of stores in each zip code
numStoresSeries = frame.groupby('Zip').count().id
#initialize an empty dataframe to store this new data
numStoresByZip = pd.DataFrame()
#populate the new dataframe with a 'zipcode' column and a 'numStores' column
numStoresByZip['Zip'] = [str(i) for i in numStoresSeries.index]
numStoresByZip['numStores'] = numStoresSeries.values

#initialize the LA County map
laMap = folium.Map(location=[34.0522,-118.2437], tiles='Stamen Toner', zoom_start=9)
folium.GeoJson(laArea).add_to(laMap)


#draw the choropleth map. These are the key components:
#--geo_path: the geojson which you want to draw on the map [in our case it is the zipcodes in LA County]

#--data: the pandas dataframe which contains the zipcode information 
# AND the values of the variable you want to plot on the choropleth

#--columns: the columns from the dataframe that you want to use 
#[this should include a geospatial column [zipcode] and a variable [numStores]

#--key_on: the common key between one of your columns and an attribute in the geojson. 
#This is how python knows which dataframe row matches up to which zipcode in the geojson

laMap.choropleth(geo_path='usmap.geojson', data=numStoresByZip, columns=['Zip', 'numStores'], \
                 key_on='feature.properties.Zip', fill_color='YlGn', fill_opacity=1)

laMap.save('laChoropleth.html')

AttributeError: 'DataFrame' object has no attribute 'id'

In [ ]:
laMap = folium.Map(location=[34.0522,-118.2437], tiles='Stamen Toner', zoom_start=9)

#add the shape of LA County to the map
folium.GeoJson(laArea).add_to(laMap)

#for each row in the Starbucks dataset, plot the corresponding latitude and longitude on the map
for i,row in df.iterrows():
    folium.CircleMarker((row.latitude,row.longitude), radius=3, weight=2, color='red', fill_color='red', fill_opacity=.5).add_to(laMap)

#save the map as an html    
laMap.save('laPointMap.html')